# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
10.13828525999999
GPU (s):
0.19059632200000465
GPU speedup over CPU: 53x


**INTIAL WORKSPACE SETUP:**



**1. FIRST GO TO EDIT >> NOTEBOOK SETTINGS >> SELECT HARDWARE ACCELERATOR AS GPU >> SAVE **


**2. THEN PRESS "CONNECT" ON RIGHT TOP TOOLBAR TO CONNECT TO SESSION. THIS SESSION WILL LAST 12 HOURS. JUST TO BE ON THE SAFE SIDE, DONT DICONNECT COMPUTER FROM NETWORK OR CLOSE THE BROWSER**

**3. AFTER CONNECTING, SELECT FILES TAB FROM LEFT PANEL TO SEE YOUR FILE SYSTEM**

In [0]:
!git clone https://github.com/kaustubh1997/ssd-mobilenet.git

Cloning into 'ssd-mobilenet'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 55 (delta 13), reused 49 (delta 10), pack-reused 0
Unpacking objects: 100% (55/55), done.


In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib

In [0]:
!git clone https://github.com/tensorflow/models.git

In [0]:
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

In [0]:
cd /content/models/research

In [0]:
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

In [0]:
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
!pip install 'prompt-toolkit==1.0.15'

In [0]:
!python object_detection/builders/model_builder_test.py

......................
----------------------------------------------------------------------
Ran 22 tests in 0.120s

OK


*The above line should display "OK". This indicates that your TensorFlow API setup is successful*

**Now copying our dataset tfrecords and config files to object_detection folder**

In [0]:
cd /content

In [0]:
cp -r ssd-mobilenet/ssd_mobilenet_v1_coco_2018_01_28/ models/research/object_detection/

In [0]:
cp -r ssd-mobilenet/training/ models/research/object_detection/

*Note: for the next step, replace "data_v3" with latest data _v folder*

In [0]:
cp -r ssd-mobilenet/data_v3/data/ models/research/object_detection/

*You can now navigate to models/research/object_detection/ from the Files Tab to the left to see that the folders have been copied into that directory*

In [0]:
cd models/research/object_detection/

**Now we will run model_main.py for training the model on our dataset:**

---

**In models/research/object_detection/training (in Files Tab), you can see event logs and model meta files being created and added as the process goes on. The training takes ~20 - 30 minutes**

*The code will run for 5000 steps. If required to run for more/less steps, change the arguments passed accordingly*

In [0]:
!python3 model_main.py --pipeline_config_path=training/ssd_mobilenet_v1_pets.config --model_dir=training --num_train_steps=5000 --sample_1_of_n_eval_examples=1 --alsologtostderr

**After training is completed, you may choose to zip the entire training directory and send it to your google drive, from where you can reliably download it and extract it to the same location (models/research/object_detection/training) on your computer**

---

*After extracting, you will have to run export_inference_graph.py on the checkpoint you want to convert to the inference graph. There are usually 2-3 checkpoints saved at various intervals from 1000-5000 training steps*

In [0]:
!zip -r training.zip training

In [0]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()

In [0]:
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
   file_metadata = {'name': name,'mimeType': 'application/octet-stream'}

   media = MediaFileUpload(path, mimetype='application/octet-stream',resumable=True)

   created = drive_service.files().create(body=file_metadata,media_body=media,fields='id').execute()

   print('File ID: {}'.format(created.get('id')))

   return created

In [0]:
save_file_to_drive('training_v0.zip', 'training.zip')

**Go to Drive, download your .zip from there**

---

After unpacking into required directory, you can run Tensorboard in your terminal at models/research/object_detection/ to see the results of your training

In [0]:
tensorboard --logdir='training'

*Open tensorboard on your web browser by going to http://localhost:6006/*

---

*Additionally, after unpacking into required directory, run the following command on you computer to get inference graph there. Running it here will create a inference graph there*

*Note: change '"78" in the command line argument with the number of the checkpoint file which you want.*

In [0]:
!python3 export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix training/model.ckpt-78 output_directory square_out

**GREAT! Now you have a frozen inference graph for you checkpoint in a folder called square_out. You can load this in the ** *object_detection_tutorial.ipynb* **notebook, and test out images on the model (Do this on your PC)**